In [62]:
import pandas as pd
import numpy as np
import pickle
import os
import glob
from datetime import datetime as dt
%matplotlib inline

from sklearn.model_selection import train_test_split

pd.set_option("display.max_colwidth",999)
pd.set_option("display.max_rows",999)
pd.set_option("display.max_columns",999)

## for all csv's in the SF folder, concatenate into one dataframe

In [63]:
files = glob.glob('SF/*.gz')
df_sf = pd.concat([pd.read_csv(fp, parse_dates = ['calendar_last_scraped','last_scraped', 'first_review', 'last_review']) for fp in files], ignore_index=True)

/home/ubuntu/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2903: DtypeWarning: Columns (43) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


## Drop the rows where the number of reviews is greater than 0 and the reviews per month is zero

In [64]:
df_sf = df_sf[(df_sf['reviews_per_month'].notnull()) | ((df_sf['number_of_reviews']== 0) & (df_sf['reviews_per_month'].isnull()))]

## fill 0 in the reviews per month where total reviews = 0

In [65]:
df_sf.reviews_per_month.fillna(value=0, inplace=True)

## create year and month columns

In [66]:
df_sf['year'] = df_sf.calendar_last_scraped.apply(lambda x: x.year)
df_sf['month'] = df_sf.calendar_last_scraped.apply(lambda x: x.month)

## Fill nulls

In [68]:
df_sf.cancellation_policy.fillna(value='none', inplace=True)
df_sf.cleaning_fee.fillna(value='$0.00', inplace=True)

df_sf.loc[df_sf.host_about.notnull(), 'host_about_filled'] = True
df_sf.loc[df_sf.host_about.isnull(), 'host_about_filled'] = False
df_sf.loc[df_sf.host_picture_url.notnull(), 'host_picture_url_filled'] = True
df_sf.loc[df_sf.host_picture_url.isnull(), 'host_picture_url_filled'] = False

df_sf['price'].replace('[\$,]', '', regex=True, inplace=True)
df_sf['extra_people'].replace('[\$,]', '', regex=True, inplace=True)

## 2017 dataframe

In [7]:
df_sf_2017 = df_sf[df_sf['year']==2017]

## 2.57 is the cutoff reviews per month to be in the top 20% of listings for all of SF in 2017

In [10]:
# df_sf.loc[df_sf.reviews_per_month >= 2.54, 'popular'] = True
# df_sf.loc[df_sf.reviews_per_month < 2.54, 'popular'] = False

In [69]:
df_sf_2017.loc[df_sf_2017.reviews_per_month >= 2.57, 'popular'] = True
df_sf_2017.loc[df_sf_2017.reviews_per_month < 2.57, 'popular'] = False

In [70]:
df_sf_2017.popular.value_counts(1)

False    0.799973
True     0.200027
Name: popular, dtype: float64

## pickle the two dataframes

In [71]:
df_sf.to_pickle('data_sf.p')
df_sf_2017.to_pickle('data_sf_2017.p')